# 계산 노예

In [18]:
import sys

print(sys.executable)

/usr/bin/python3


In [19]:
import sympy as sp
import custom_sympy as csp

In [20]:
# 시간 정의
t = sp.symbols("t", real=True)

# 상수 정의
m, M, l, g = sp.symbols("m M l g", real=True)
fric_x, fric_theta = sp.symbols("fric_x fric_theta", real=True)


# 차량에 대한 외력 정의
f = sp.symbols("f", real=True)

# state 변수 정의
x = sp.Function("x")(t)
theta = sp.Function("theta")(t)
x_dot = x.diff(t)
theta_dot = theta.diff(t)
x_ddot = x_dot.diff(t)
theta_ddot = theta_dot.diff(t)

state = csp.make_state_vector([(x, 1), (theta, 1)], t)


# 각 질량체의 x,y 좌표 정의
cart_pos_x = x
cart_pos_y = 0
pendulum_pos_x = x + l * sp.sin(theta)
pendulum_pos_y = l * sp.cos(theta)

# 각 질량체의 속도의 제곱
cart_vel_sqare = cart_pos_x.diff(t) ** 2
pendulum_vel_sqare = pendulum_pos_x.diff(t) ** 2 + pendulum_pos_y.diff(t) ** 2

# 시스템의 총 위치에너지와 운동에너지
V = m * g * pendulum_pos_y  # 위치에너지
T = 1 / 2 * M * cart_vel_sqare + 1 / 2 * m * pendulum_vel_sqare  # 운동에너지

# 라그랑지안
L = T - V
L.simplify()

# 각 좌표계에 대한 라그랑주 방정식. 마찰력과 외력을 포함한다.
x_eq = sp.Eq(L.diff(x_dot).diff(t) - L.diff(x), f).simplify()
theta_eq = sp.Eq(L.diff(theta_dot).diff(t) - L.diff(theta), 0).simplify()

solution = sp.solve([x_eq, theta_eq], [x_ddot, theta_ddot])

sol_x_ddot = solution[x_ddot].simplify()
sol_theta_ddot = solution[theta_ddot].simplify()

state_dot = state.diff(t).subs([(x_ddot, sol_x_ddot), (theta_ddot, sol_theta_ddot)])

csp.make_state_comfy_to_see(sp.Eq(state.diff(t), state_dot))

Eq(Matrix([
[     xdot],
[    xddot],
[ thetadot],
[thetaddot]]), Matrix([
[                                                                                                       xdot],
[                                (f - g*m*sin(2*theta)/2 + l*m*thetadot**2*sin(theta))/(M + m*sin(theta)**2)],
[                                                                                                   thetadot],
[(M*g*sin(theta) - f*cos(theta) + g*m*sin(theta) - l*m*thetadot**2*sin(2*theta)/2)/(l*(M + m*sin(theta)**2))]]))

f(x)를 찾아낸다.

In [21]:
# f(x)
f_x = state_dot.subs(f, 0)
f_x = sp.simplify(f_x)
csp.make_state_comfy_to_see(f_x)

Matrix([
[                                                                         xdot],
[           m*(-g*cos(theta) + l*thetadot**2)*sin(theta)/(M + m*sin(theta)**2)],
[                                                                     thetadot],
[(M*g + g*m - l*m*thetadot**2*cos(theta))*sin(theta)/(l*(M + m*sin(theta)**2))]])

g(x)를 찾아낸다.

In [22]:
g_x = (state_dot - f_x) / f
g_x = sp.simplify(g_x)
csp.make_state_comfy_to_see(g_x)

Matrix([
[                                    0],
[              1/(M + m*sin(theta)**2)],
[                                    0],
[-cos(theta)/(l*(M + m*sin(theta)**2))]])

## HO-CBF 만들기

2차 order CBF를 만들 것이다. 이것의 식은 다음과 같다.

$$\ddot{h}+\alpha(\dot{h}) \geq 0$$


응~ 틀렸어! 찬찬히 봐봐. 우선, $\alpha(x) = k x$라고 하자.
$$\begin{aligned}
\psi_0(\mathbf{x}) &:= h(\mathbf{x}) \\
\psi_1(\mathbf{x}) &:= \dot{\psi}_0(\mathbf{x}) + k_1 \psi_0(\mathbf{x}) \\
&= \dot{h}(\mathbf{x}) + k_1 h(\mathbf{x}) \\
\psi_2(\mathbf{x}) &:= \dot{\psi}_1(\mathbf{x}) + k_2 \psi_1(\mathbf{x}) \\
&= \ddot{h}(\mathbf{x}) + k_2 \dot{h}(\mathbf{x}) + k_2 k_1 h(\mathbf{x})
\end{aligned}$$

그러니까 부등식은 다음과 같아야해.
$$\ddot{h}(\mathbf{x}) + k_2 \dot{h}(\mathbf{x}) + k_2 k_1 h(\mathbf{x}) \geq 0 $$

우선 $h(x)$를 정의한다.

In [84]:
x_min = sp.symbols("x_min", real=True)
x_max = sp.symbols("x_max", real=True)

h_x = -(state[0] - x_min) * (state[0] - x_max)

h_x

(-x_max + x(t))*(x_min - x(t))

$\frac{dh}{d\mathbf{x}}$ 계산

In [85]:
dh_dstate = h_x.diff(state).T
dh_dstate

Matrix([[x_max + x_min - 2*x(t), 0, 0, 0]])

$\dot{h}$를 구한다.

In [86]:
h_dot_x = csp.make_11matrix_to_scalar(dh_dstate @ f_x + dh_dstate @ g_x)
csp.make_state_comfy_to_see(h_dot_x)

xdot*(-2*x + x_max + x_min)

그 다음 

$$\ddot{h}=\frac{d\dot{h}}{d\mathbf{x}}f(\mathbf{x}) + \frac{d\dot{h}}{d\mathbf{x}}g(\mathbf{x})$$

이므로 $\frac{d\dot{h}}{d\mathbf{x}}$를 구한다.

In [87]:
dh_dot_x_dstate = h_dot_x.diff(state).T
csp.make_state_comfy_to_see(dh_dot_x_dstate)

Matrix([[-2*xdot, -2*x + x_max + x_min, 0, 0]])

따라서 $\ddot{h}$는 다음과 같다.

In [88]:
h_ddot_x = dh_dot_x_dstate @ f_x + dh_dot_x_dstate @ g_x * f

h_ddot_x = csp.make_11matrix_to_scalar(h_ddot_x)

csp.make_state_comfy_to_see(h_ddot_x)

(f*(-2*x + x_max + x_min) - m*(g*cos(theta) - l*thetadot**2)*(-2*x + x_max + x_min)*sin(theta) - 2*xdot**2*(M + m*sin(theta)**2))/(M + m*sin(theta)**2)

$$\ddot{h}(\mathbf{x}) + k_2 \dot{h}(\mathbf{x}) + k_2 k_1 h(\mathbf{x}) \geq 0$$ 
이걸 구현하자.

In [89]:
k_1, k_2 = sp.symbols("k_1 k_2", real=True)
u = sp.symbols("u", real=True)
ineq = (h_ddot_x + k_2 * h_dot_x + k_2 * k_1 * h_x >= 0).simplify().subs(f, u)
csp.make_state_comfy_to_see(ineq)

(k_2*(M + m*sin(theta)**2)*(-k_1*(-x + x_max)*(-x + x_min) + xdot*(-2*x + x_max + x_min)) - m*(g*cos(theta) - l*thetadot**2)*(-2*x + x_max + x_min)*sin(theta) + u*(-2*x + x_max + x_min) - 2*xdot**2*(M + m*sin(theta)**2))/(M + m*sin(theta)**2) >= 0

분모가 항상 0보다 크다. 분모 제거

In [90]:
D = M + m * sp.sin(theta) ** 2
ineq1 = ineq.lhs * D >= 0
ineq1 = ineq1.simplify()
csp.make_state_comfy_to_see(ineq1)

k_2*(M + m*sin(theta)**2)*(k_1*(-x + x_max)*(-x + x_min) - xdot*(-2*x + x_max + x_min)) + m*(g*cos(theta) - l*thetadot**2)*(-2*x + x_max + x_min)*sin(theta) - u*(-2*x + x_max + x_min) + 2*xdot**2*(M + m*sin(theta)**2) <= 0

$a(\mathbf{x}) u \leq b(\mathbf{x})$꼴로 고친다.

In [91]:
ineq2 = sp.solve(ineq1, u).simplify()
csp.make_state_comfy_to_see(ineq2)

u*(2*x - x_max - x_min) <= -M*k_1*k_2*x**2 + M*k_1*k_2*x*x_max + M*k_1*k_2*x*x_min - M*k_1*k_2*x_max*x_min - 2*M*k_2*x*xdot + M*k_2*x_max*xdot + M*k_2*x_min*xdot - 2*M*xdot**2 + g*m*x*sin(2*theta) - g*m*x_max*sin(2*theta)/2 - g*m*x_min*sin(2*theta)/2 - k_1*k_2*m*x**2*sin(theta)**2 + k_1*k_2*m*x*x_max*sin(theta)**2 + k_1*k_2*m*x*x_min*sin(theta)**2 - k_1*k_2*m*x_max*x_min*sin(theta)**2 - 2*k_2*m*x*xdot*sin(theta)**2 + k_2*m*x_max*xdot*sin(theta)**2 + k_2*m*x_min*xdot*sin(theta)**2 - 2*l*m*thetadot**2*x*sin(theta) + l*m*thetadot**2*x_max*sin(theta) + l*m*thetadot**2*x_min*sin(theta) - 2*m*xdot**2*sin(theta)**2

In [92]:
u_side = ineq2.lhs.simplify()
other_side = ineq2.rhs.simplify()

In [93]:
csp.make_state_comfy_to_see(u_side)

u*(2*x - x_max - x_min)

In [94]:
csp.make_state_comfy_to_see(other_side)

-M*k_1*k_2*x**2 + M*k_1*k_2*x*x_max + M*k_1*k_2*x*x_min - M*k_1*k_2*x_max*x_min - 2*M*k_2*x*xdot + M*k_2*x_max*xdot + M*k_2*x_min*xdot - 2*M*xdot**2 + g*m*x*sin(2*theta) - g*m*x_max*sin(2*theta)/2 - g*m*x_min*sin(2*theta)/2 - k_1*k_2*m*x**2*sin(theta)**2 + k_1*k_2*m*x*x_max*sin(theta)**2 + k_1*k_2*m*x*x_min*sin(theta)**2 - k_1*k_2*m*x_max*x_min*sin(theta)**2 - 2*k_2*m*x*xdot*sin(theta)**2 + k_2*m*x_max*xdot*sin(theta)**2 + k_2*m*x_min*xdot*sin(theta)**2 - 2*l*m*thetadot**2*x*sin(theta) + l*m*thetadot**2*x_max*sin(theta) + l*m*thetadot**2*x_min*sin(theta) - 2*m*xdot**2*sin(theta)**2

값을 넣어보자.

In [95]:
dict_state = {x: 1, theta: 0, x_dot: 0, theta_dot: 0}
dict_const = {m: 0.1, M: 1, l: 1, g: 9.81, k_1: 1, k_2: 1, x_min: -2, x_max: 2}
result = ineq2.subs(dict_const).simplify()
result = result.subs(dict_state).simplify()
csp.make_state_comfy_to_see(result)

u <= 3/2